## Import Required Libraries

In [ ]:
from ERApy import ERADist
import numpy as np
import matplotlib.pyplot as plt

## Helper Function: Generate Bimodal Gaussian Data

In [ ]:
def sample_bimodal_gaussian(n_samples=1000, mix_weights=(0.4, 0.6),
                            means=(-2, 3), stds=(0.7, 1.2)):
    comps = np.random.choice([0, 1], size=n_samples, p=mix_weights)
    data = np.where(
        comps == 0,
        np.random.normal(loc=means[0], scale=stds[0], size=n_samples),
        np.random.normal(loc=means[1], scale=stds[1], size=n_samples),
    )
    return data

## Generate Sample Data

In [ ]:
np.random.seed(2025)  # initializing random number generator
n = 2000  # number of data points

# generate a bimodal Gaussian mixture dataset
data = sample_bimodal_gaussian(n_samples=n,
                               mix_weights=(0.2, 0.8),
                               means=(-2, 3),
                               stds=(0.5, 1.0))
weights = None

print(f"Generated {len(data)} data points")
print(f"Data range: [{data.min():.2f}, {data.max():.2f}]")
print(f"Data mean: {data.mean():.3f}")
print(f"Data std: {data.std():.3f}")

## Define ERADist Object with Empirical Data

In [ ]:
# Definition of an ERADist object by a dataset
dist = ERADist('empirical', 'DATA', [data, weights, "kde", None, {"bw_method": None}])

# computation of the first two moments
mean_dist = dist.mean()
std_dist = dist.std()

print(f"Empirical distribution created")
print(f"Mean: {mean_dist:.4f}")
print(f"Standard Deviation: {std_dist:.4f}")

# generation of n random samples
n_samples = 2000
samples = dist.random(n_samples)
print(f"Generated {n_samples} random samples from empirical distribution")

## Demonstrate ERADist Methods

In [ ]:
# generation of n samples x to work with
x = dist.random(n_samples)
print(f"Generated {len(x)} test samples")
print(f"Sample values (first 5): {x[:5]}")

# computation of the PDF for the samples x
pdf = dist.pdf(x)
print(f"PDF values (first 5): {pdf[:5]}")

# computation of the CDF for the samples x
cdf = dist.cdf(x)
print(f"CDF values (first 5): {cdf[:5]}")

# computation of the inverse CDF based on the CDF values (-> initial x)
icdf = dist.icdf(cdf)
print(f"Inverse CDF values (first 5): {icdf[:5]}")
print(f"Reconstruction error (max): {np.max(np.abs(x - icdf)):.6f}")

## Plot PDF and CDF

In [ ]:
# Plot of the PDF and CDF
x_plot = np.linspace(data.min()-1, data.max()+1, 200)  # values for which the PDF and CDF are evaluated
pdf = dist.pdf(x_plot)  # computation of PDF
cdf = dist.cdf(x_plot)  # computation of CDF

fig_dist = plt.figure(figsize=[10, 10])

fig_pdf = fig_dist.add_subplot(211)
fig_pdf.hist(data, density=True, bins=int(np.sqrt(len(data))), label="data", color='r', alpha=0.3)
fig_pdf.plot(x_plot, pdf, 'b', lw=2, label="Empirical PDF")
fig_pdf.set_xlim([data.min(), data.max()])
fig_pdf.set_xlabel(r'$X$')
fig_pdf.set_ylabel(r'$PDF$')
fig_pdf.set_title('Empirical Distribution - PDF')
fig_pdf.legend()
fig_pdf.grid(True, alpha=0.3)

fig_cdf = fig_dist.add_subplot(212)
fig_cdf.plot(x_plot, cdf, "b", lw=2, label="Empirical CDF")
fig_cdf.set_xlim([data.min(), data.max()])
fig_cdf.set_xlabel(r'$X$')
fig_cdf.set_ylabel(r'$CDF$')
fig_cdf.set_title('Empirical Distribution - CDF')
fig_cdf.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Empirical ERADist example completed!")